In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
sys.path.append("../..")
sys.path.append("..")
sys.path.append(os.getcwd())
import re
import datetime

import math
import scipy

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import seaborn as sns
sns.set()
sns.set(palette="bright")
sns.set_style("whitegrid")

from sklearn import linear_model
from cycler import cycler
from celluloid import Camera
from IPython.display import HTML

import importlib
import warnings
import numpy as np
import pandas as pd
import copy
import random
from tslib.src import tsUtils
from tslib.tests import testdata
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, DBSCAN #For clustering
from sklearn.mixture import GaussianMixture #For GMM clustering
from filter_data import *
from load_and_clean import *
import hdbscan
import plotly.figure_factory as ff
%matplotlib inline
import json

In [3]:
temp_data, fips = load_clean('temperature')
post_memo = temp_data.loc['2020-05-25':]

In [ ]:
plt.figure(figsize = (15, 12))
plt.scatter(post_memo.mean(), post_memo.std())
plt.show()

## HDBSCAN

In [ ]:
feature_dict, features = cluster_temperature(post_memo)

In [ ]:
df = pd.DataFrame.drop_duplicates(pd.merge(features['DB'].reset_index(), fips, 
                              how = 'right', on = ['county_state']))
group = df['fips']
values = df['DB']
fig = ff.create_choropleth(fips=group, values=values)
fig.layout.template = None
fig.show()

In [ ]:
post_memo[feature_dict[1]].plot(figsize = (15, 10))

## K-means

In [ ]:
#k-means
feature_dict, features = cluster_temperature(post_memo, cluster_method = 'kmeans')

In [ ]:
df = pd.DataFrame.drop_duplicates(pd.merge(features['kmeans'].reset_index(), fips, 
                              how = 'right', on = ['county_state']))
group = df['fips']
values = df['kmeans']
fig = ff.create_choropleth(fips=group, values=values)
fig.layout.template = None
fig.show()

In [ ]:
for key in feature_dict:
    feature_dict[key] = list(feature_dict[key])
np.save('temp_cluster.npy', feature_dict) 

In [ ]:
json = json.dumps(feature_dict)
f = open("temp_cluster.json","w")
f.write(json)
f.close()

## AgglomerativeClustering

In [ ]:
feature_dict, features = cluster_temperature(post_memo, cluster_method = 'AgglomerativeClustering')

In [ ]:
df = pd.DataFrame.drop_duplicates(pd.merge(features['AC'].reset_index(), fips, 
                              how = 'right', on = ['county_state']))
group = df['fips']
values = df['AC']
fig = ff.create_choropleth(fips=group, values=values)
fig.layout.template = None
fig.show()

In [ ]:
for key in feature_dict:
    feature_dict[key] = list(feature_dict[key])
np.save('temp_cluster.npy', feature_dict)